In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///../Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(autoload_with=engine)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
most_recent_date = session.query(func.max(Measurement.date)).scalar()

#convert to date time obj
most_recent_date = dt.datetime.strptime(most_recent_date, '%Y-%m-%d').date()
print(most_recent_date)

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database, 
# Calculate the date one year from the last date in data set.

one_year_ago = most_recent_date - dt.timedelta(days=365)

# Perform a query to retrieve the data and precipitation scores
precipitation_data = session.query(Measurement.date, Measurement.prcp)\
    .filter(Measurement.date >= one_year_ago)\
    .filter(Measurement.date <= most_recent_date)\
    .all()
print(precipitation_data)



In [ ]:
# Save the query results as a Pandas DataFrame. Explicitly set the column names
precipitation_df = pd.DataFrame(precipitation_data, columns=['Date', 'Precipitation'])

# Sort the dataframe by date
precip_date_df = precipitation_df.sort_values('Date', ascending=True
                                            )

# Use Pandas Plotting with Matplotlib to plot the data
ax = precip_date_df.plot(x='Date', y='Precipitation', figsize=(20, 10))

# Set the x-axis label
ax.set_xlabel('Date')

# Set the y-axis label
ax.set_ylabel('Precipitation (inches)')

# Set the plot title
ax.set_title('Precipitation in the Last 12 Months')

# Get the x-tick positions and labels
tick_positions = range(len(precip_date_df))
tick_labels = precip_date_df['Date']

# Set the x-tick positions and labels
plt.xticks(tick_positions[::30], tick_labels[::30], rotation='vertical')

plt.legend(loc='upper center')
plt.tight_layout()
plt.show()

In [ ]:
# Calculate summary statistics for precipitation data
precipitation_stats = precipitation_df['Precipitation'].describe()

# Print the summary statistics
print(precipitation_stats)

Count: The number of non-missing values in the precipitation data. In this case, there are 2021 data points.

Mean: The average (arithmetic mean) of the precipitation values. The mean is approximately 0.177279, indicating that on average, the recorded precipitation is around 0.177279 inches.

STD: The standard deviation of the precipitation values. The standard deviation is approximately 0.461190, which measures the variability or spread of the data points around the mean. A higher standard deviation suggests a greater dispersion of values.

Min: The minimum value in the precipitation data. The minimum precipitation recorded is 0 inches.

25%: The 25th percentile or the first quartile. This value represents the cutoff below which 25% of the data points fall. In this case, 25% of the precipitation values are below 0 inches.

50%: The 50th percentile or the median. This is the middle value in the sorted precipitation data. In this case, the median precipitation is approximately 0.02 inches, meaning that 50% of the data points are below this value.

75%: The 75th percentile or the third quartile. This value represents the cutoff below which 75% of the data points fall. In this case, 75% of the precipitation values are below 0.13 inches.

Max: The maximum value in the precipitation data. The maximum recorded precipitation is 6.7 inches.

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset
station_count = session.query(func.count(Station.station)).scalar()

print(station_count)

In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
station_activity = session.query(Measurement.station, func.count(Measurement.station)) \
    .group_by(Measurement.station) \
    .order_by(func.count(Measurement.station).desc()) \
    .all()

# List the stations and their counts in descending order.
for station, count in station_activity:
    print(station, count)

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
active_station = 'USC00519281'


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
temperature_data = session.query(Measurement.tobs)\
    .filter(Measurement.station == active_station)\
    .filter(Measurement.date >= one_year_ago)\
    .filter(Measurement.date <= most_recent_date)\
    .all()

# Extract the temperature values from the query results
temperatures = [temp[0] for temp in temperature_data]

# Plot the histogram
plt.figure(figsize=(8, 6))
plt.hist(temperatures, bins=12, alpha=0.7)
plt.xlabel('Temperature (°F)')
plt.ylabel('Frequency')
plt.title('Temperature Observation Data (Last 12 Months)')
plt.show()

In [ ]:
# Calculate the lowest, highest, and average temperature for the most active station
temperature_stats = session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)) \
    .filter(Measurement.station == active_station) \
    .all()

# Extract the results
lowest_temp, highest_temp, avg_temp = temperature_stats[0]

# Print the results
print(f"Lowest Temperature: {lowest_temp}")
print(f"Highest Temperature: {highest_temp}")
print(f"Average Temperature: {avg_temp}")

# Close Session

In [ ]:
# Close Session
session.close()